### Load libraries

In [1]:

import sys
sys.path.insert(0, '../src/')

from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model
from load_modify_sample_utils import sample_dingo, sample_optgp

from correlations_utils import correlated_reactions, split_forward_reverse
from correlations_utils import find_reactants_products, sharing_metabolites_square_matrix

from graphs_utils import construct_graph, plot_graph, compare_betweenness_centralities

from pathways_utils import sort_reactions_in_pathways_by_reactions_in_model_order, subset_sampling_df_from_reaction_ids
from pathways_utils import map_model_to_kegg_reactions, read_json_file, bigg_to_kegg_id, fill_missing_kegg_ids_in_dict
from pathways_utils import get_kegg_pathways_from_reaction_ids, subset_model_reactions_from_pathway_info
from pathways_utils import dictionary_bigg_id_to_pathway_names, dictionary_forward_reverse_bigg_id_to_pathway_names


/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/touliopoulos/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Load model

In [2]:
ec_cobra_model, ec_cobra_reactions, ec_dingo_model, ec_dingo_reactions = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)

Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.

### Modify model and sample

In [3]:

ec_cobra_model_condition_100, ec_dingo_model_condition_100 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=100)
ec_cobra_model_condition_0, ec_dingo_model_condition_0 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=0)


samples_optgp_condition_100 = sample_optgp(ec_cobra_model_condition_100, n_samples = 3000, thinning=100, reaction_in_rows = True)
samples_optgp_condition_0 = sample_optgp(ec_cobra_model_condition_0, n_samples = 3000, thinning=100, reaction_in_rows = True)


Read LP format model from file /tmp/tmp3g1qdg6z.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros


Read LP format model from file /tmp/tmpuft3owht.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp7lx78k04.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmpz1ne29by.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


### Map KEGG pathway information to reactions BiGG ids  

In [4]:
initial_bigg_to_kegg_dictionary = map_model_to_kegg_reactions(ec_cobra_model)


reactions_json, reactions_pandas = read_json_file("../ext_data/reactions/reactions.json")


final_bigg_to_kegg_dictionary = fill_missing_kegg_ids_in_dict(initial_bigg_to_kegg_dictionary, reactions_pandas)


df_kegg_pathways = get_kegg_pathways_from_reaction_ids(final_bigg_to_kegg_dictionary)

WARNING [bioservices.KEGG:596]:  status is not ok with Bad Request
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.
WARNING [bioservices.KEGG:596]:  status is not ok with Bad Request
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.
WARNING [bioservices.KEGG:596]:  status is not ok with Bad Request
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.
WARNING [bioservices.KEGG:596]:  status is not ok with Bad Request
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.
WARNING [bioservices.KEGG:596]:  status is not ok with Bad Request
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.
WARNING [bioservices.KEGG:596]:  status is not ok with Bad Request
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.
WARNING [bioservices.KEGG:596]:  status is not ok with Bad Request
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.
WARNING [bioservices.KEGG:596]:  s

### Subset sampling dataframe to include only reactions from pathways of interest

In [5]:

glycolysis = subset_model_reactions_from_pathway_info(df_kegg_pathways, "Glycolysis")
ppp = subset_model_reactions_from_pathway_info(df_kegg_pathways, "Pentose")


reactions_in_pathways_ordered_duplicates = sort_reactions_in_pathways_by_reactions_in_model_order(ec_dingo_reactions, glycolysis, ppp)


# Remove duplicates
reactions_in_pathways_ordered = []
[reactions_in_pathways_ordered.append(val) for val in reactions_in_pathways_ordered_duplicates if val not in reactions_in_pathways_ordered]


subset_pathways_optgp_condition_100 = subset_sampling_df_from_reaction_ids(
                                                                samples_optgp_condition_100, 
                                                                ec_dingo_reactions, 
                                                                subset_reactions = reactions_in_pathways_ordered)


subset_pathways_optgp_condition_0 = subset_sampling_df_from_reaction_ids(
                                                                samples_optgp_condition_0, 
                                                                ec_dingo_reactions, 
                                                                subset_reactions = reactions_in_pathways_ordered)


### Split into forward and reverse reactions

In [6]:

extended_steady_states_100, extended_reactions_100 = split_forward_reverse(subset_pathways_optgp_condition_100, reactions=reactions_in_pathways_ordered)

reversibility_list_all_reactions, reactants_list_all_reactions, products_list_all_reactions = find_reactants_products(ec_cobra_model, extended_reactions_100)
print(len(reversibility_list_all_reactions), len(reactants_list_all_reactions))



extended_steady_states_0, extended_reactions_0 = split_forward_reverse(subset_pathways_optgp_condition_0, reactions=reactions_in_pathways_ordered)

reversibility_list_all_reactions, reactants_list_all_reactions, products_list_all_reactions = find_reactants_products(ec_cobra_model, extended_reactions_0)
print(len(reversibility_list_all_reactions), len(reactants_list_all_reactions))


21 21
25 25


### Create a dictionary that maps pathways from KEGG information

In [7]:

bigg_to_pathway_dict = dictionary_bigg_id_to_pathway_names(glycolysis = glycolysis, ppp = ppp)

group_map_100 = dictionary_forward_reverse_bigg_id_to_pathway_names(bigg_to_pathway_dict, extended_reactions_100)
group_map_0 = dictionary_forward_reverse_bigg_id_to_pathway_names(bigg_to_pathway_dict, extended_reactions_0)


### Calculate the correlation matrix (without sharing metabolites filtering)

In [8]:

(linear_correlation_matrix_100, 
non_linear_correlation_matrix_100, 
mixed_correlation_matrix_100, 
correlations_dictionary_100) = correlated_reactions(
        steady_states = extended_steady_states_100,
        boolean_sharing_metabolites_matrix=None,
        reactions=extended_reactions_100, 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True)



(linear_correlation_matrix_0, 
non_linear_correlation_matrix_0, 
mixed_correlation_matrix_0, 
correlations_dictionary_0) = correlated_reactions(
        steady_states = extended_steady_states_0,
        boolean_sharing_metabolites_matrix=None,
        reactions=extended_reactions_0, 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 4, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True)


Completed the process of 30 from 103 copulas
Completed the process of 31 from 103 copulas
Completed the process of 32 from 103 copulas
Completed the process of 33 from 103 copulas
Completed the process of 34 from 103 copulas
Completed the process of 35 from 103 copulas
Completed the process of 44 from 103 copulas
Completed the process of 49 from 103 copulas
Completed the process of 54 from 103 copulas
Completed the process of 59 from 103 copulas
Completed the process of 64 from 103 copulas
Completed the process of 69 from 103 copulas
Completed the process of 74 from 103 copulas
Completed the process of 96 from 103 copulas
Completed the process of 102 from 103 copulas
Completed the process of 1 from 144 copulas
Completed the process of 2 from 144 copulas
Completed the process of 3 from 144 copulas
Completed the process of 4 from 144 copulas
Completed the process of 5 from 144 copulas
Completed the process of 6 from 144 copulas
Completed the process of 7 from 144 copulas
Completed the pr

### Create and plot a graph from each condition

In [10]:


G100, pos100 = construct_graph(linear_correlation_matrix_100,
                         non_linear_correlation_matrix_100,
                         reactions=extended_reactions_100,
                         remove_unconnected_nodes=False,
                         correction=False,
                         group_map=group_map_100)

plot_graph(G100, pos100)



G0, pos0= construct_graph(linear_correlation_matrix_0, 
                         non_linear_correlation_matrix_0, 
                         reactions=extended_reactions_0,
                         remove_unconnected_nodes=False,
                         correction=False,
                         group_map=group_map_0)

plot_graph(G0, pos0)


### Metrics from graph analysis that compare (a) graphs occured from different conditions (modularity) and (b) nodes centralities/importance in the network

In [ ]:


from graphs_utils import compare_network_modularities, compare_betweenness_centralities, compare_closeness_centralities, weighted_node_centrality


modularities_diff = compare_network_modularities(G100, G0)
print(modularities_diff)


sorted_nodes = compare_betweenness_centralities(G100, G0)
print(sorted_nodes)


sorted_nodes_2 = compare_closeness_centralities(G100, G0)
print(sorted_nodes_2)


sorted_nodes_3 = weighted_node_centrality(G100, G0)
print(sorted_nodes_3)




3.3306690738754696e-16 7.771561172376096e-16
Increased Modularity: Graph_1
4.440892098500626e-16
[('TPI_rev', 0.9353825501120967), ('TKT1_rev', 0.7292430853744388), ('PGK_rev', 0.37264723995109406), ('TKT1', 0.2590160992315603), ('G6PDH2r', 0.18175008356712108), ('FBA_rev', 0.17703198737005366), ('PGK', 0.14830245727527674), ('PGI_rev', 0.07072765481081911), ('TKT2_rev', 0.05756884585251726), ('RPE_rev', 0.026039870890885088), ('TALA_rev', 0.018887850336126197), ('TKT2', 0.0021842304679018675), ('PFK', 0.0), ('PYK', 0.0), ('PGL', -0.0014629303740875954), ('PGM', -0.021915529564652363), ('ENO', -0.021915529564652363), ('PGM_rev', -0.021915529564652363), ('GND', -0.045623164228902724), ('RPE', -0.046303209031142205), ('FBA', -0.12470711307258056), ('TPI', -0.126448388147699), ('RPI', -0.14376779801428896), ('PGI', -0.22603647774278707), ('TALA', -0.24396494355054552), ('GAPD', -0.2470002258120118), ('RPI_rev', -0.43493289023201287)]
[('PGI_rev', 18.852835539822582), ('TKT1_rev', 18.85280